# System Information

Autogenerated info about each system

In [ ]:
# Plotting setup
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import ticker
from IPython.display import display, display_markdown

import pandas as pd
import os, sys
from pprint import pprint
sys.path.extend(('../../reframe', '../../')) # the `modules` package imports `reframe` so need that in sys.modules
import modules

import json, pprint
from collections import defaultdict

pd.options.display.max_columns = None
pd.options.display.max_colwidth = None

In [ ]:
 # load all sysinfo.json files:
sysinfos = {} # key-> reframe "system:partition", value-> nested dict of values
for path in modules.utils.find_run_outputs(root='../../output', test='Sysinfo', ext='.json'):
    # load metadata:
    meta = modules.utils.parse_path_metadata(path)
    syspart = '%s:%s' % (meta['sysname'], meta['partition']) # throw away environment
    with open(path) as f:
        sysinfos[syspart] = json.load(f)

# System-level Information

In [ ]:
# Derive and tabulate calculated values:
general_table = []
general_cols = ['Number of nodes', 'Total CPUs', 'Total memory']
for syspart, sysinfo in sysinfos.items():
    
    num_nodes = len([hostdata['hostname'] for hostdata in sysinfo.values()])
    
    total_cpus = sum(int(modules.utils.get_nested(hostdata, 'cpu.CPU(s)')) for hostdata in sysinfo.values())
    
    mems = [modules.utils.get_nested(hostdata, 'memory.total') for hostdata in sysinfo.values()]
    mem_units = list(set(modules.utils.split_numeric(m)[1] for m in mems))
    if len(mem_units) > 1:
        raise NotImplementedError('Cannot cope with different units for memory size across cluster: %r' % mems)
    total_mem = '%s %s' % (sum(int(modules.utils.split_numeric(m)[0]) for m in mems), mem_units[0])
    
    general_table.append([num_nodes, total_cpus, total_mem])
    general_df = pd.DataFrame(general_table, index=list(sysinfos.keys()), columns=general_cols, dtype=str)
    display(general_df)

# Node Details

In [ ]:
# Define general tables:
TABLES = {'Operating system': {'OS':'os.release.PRETTY_NAME', 'Kernel':'os.kernel'},
          'Chassis': {'model':'chassis.product_name', 'vendor':'chassis.sys_vendor'},
          'CPU': {'architecture':'cpu.Architecture', 'model':'cpu.Model name',
                  'cpus /node':'cpu.CPU(s)', 'sockets /node':'cpu.Socket(s)', 'cores /socket':'cpu.Core(s) per socket',
                  'threads /core':'cpu.Thread(s) per core',
                 },
          'Memory': {'memory /node':'memory.total', 'type':'memory.types'},
         }
    
# Group system info across reframe partitions and tabulate:
for table_title, table_contents in TABLES.items():
    table = {}
    for syspart, sysinfo in sysinfos.items():
        table[syspart] = {}
        for table_label, datakey in table_contents.items():
            for hostname, nodedata in sysinfo.items():
                val = modules.utils.get_nested(nodedata, datakey)
                table[syspart].setdefault(table_label, set()).add(val)
                
    display_markdown('### %s' % table_title, raw=True)
    df = pd.DataFrame(table)
    df = df.applymap(modules.utils.singleval)
    display(df.transpose())

In [ ]:
IFACE_DETAILED_PARAMS = ['features', 'pause_opts', 'ring_max', 'ring_curr']

# Gather network data: this is a bit different in that sysinfo['net'] -> {iface1:{parameters..}, iface2:{parameters...}, ...}    
netvals = defaultdict(dict)
for syspart, sysinfo in sysinfos.items():
    for hostname, nodedata in sysinfo.items():
        for iface in nodedata['net']:
            for k, v in nodedata['net'][iface].items():
                pw = '%s: %s' % (syspart, iface)
                if isinstance(v, list):
                    v = ','.join(v)
                netvals[pw].setdefault(k, set()).add(v)
                
df = pd.DataFrame(netvals)
df = df.applymap(modules.utils.singleval)
df = df.transpose()

display_markdown('### Network adaptors - general information', raw=True)
general_df = df[df.columns.difference(IFACE_DETAILED_PARAMS)]
display(general_df)

display_markdown('### Network adaptors - detailed parameters', raw=True)
details_df = df[IFACE_DETAILED_PARAMS]
display(details_df)

In [ ]:
# # just to show which way around we want it:

# example = [['128GB', '24'],
#            ['256GB', '128'],
#            ['64GB', '6']
#           ]

# index = ['medium sys', 'large sys', 'small sys']
# cols = ['memory', 'num nodes']
# df = pd.DataFrame(example, index=index, columns=cols, dtype=str)
# display(df)